In [1]:
import os
#import dill
!python3 -m pip install dill
!python3 -m pip install seaborn
!python3 -m pip install verta.integrations.sklearn
!python3 -m pip install alibi
#import verta.integrations.sklearn
from alibi.explainers import AnchorTabular
# os.environ["MODIN_ENGINE"] = "ray"


You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement verta.integrations.sklearn (from versions: none)
ERROR: No matching distribution found for verta.integrations.sklearn
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
# import pandas as pd
# import modin.pandas as pd
from datetime import datetime
!python3 -m pip install watermark
import watermark
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!python3 -m pip install minio
from minio import Minio
!python3 -m pip install verta
from verta import Client
!python3 -m pip install minio.error
#from minio.error import ResponseError
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline


# import tools as tools
%matplotlib inline
%load_ext watermark

You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 818 kB 17.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement minio.error (from versions: none)
ERROR: No matching distribution found for minio.error
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


In [3]:
%watermark -n -v -m -g -iv


seaborn    0.11.1
matplotlib 3.3.4
watermark  2.0.2
pandas     1.1.5
numpy      1.19.5
Thu Jun 10 2021 

CPython 3.6.8
IPython 7.16.1

compiler   : GCC 8.3.1 20191121 (Red Hat 8.3.1-5)
system     : Linux
release    : 4.18.0-193.13.2.el8_2.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   :


In [4]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurn"
experiment_id = experiment_name + timestampStr


def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient

def get_verta():
    client = Client("http://v1-webapp:3000")
    return client

def get_meta_store():
    client = get_verta()
    proj = client.set_project("ml-workshop")
    client.set_experiment(experiment_name)
    run = client.set_experiment_run(experiment_id)
    return run




In [5]:
minioClient = get_s3_server()
data_file = minioClient.fget_object("data", "full_data_csvrachna/part-00000-ac6029ea-7ac8-4108-829d-bf8a5f4db9b1-c000.csv", "/tmp/data.csv")
data_file_version = data_file.version_id
data = pd.read_csv('/tmp/data.csv')
data.head(5)



customerID PhoneService     MultipleLines InternetService  \
0         148          Yes                No             DSL   
1         463          Yes               Yes     Fiber optic   
2         471          Yes                No              No   
3         496           No  No phone service             DSL   
4         833          Yes                No             DSL   

        OnlineSecurity         OnlineBackup     DeviceProtection  \
0                   No                   No                   No   
1                   No                   No                  Yes   
2  No internet service  No internet service  No internet service   
3                   No                  Yes                  Yes   
4                  Yes                  Yes                  Yes   

           TechSupport          StreamingTV      StreamingMovies  ...  \
0                   No                   No                   No  ...   
1                   No                  Yes                  Yes  ...   
2  No internet service  No internet service  No internet service  ...   
3                   No                   No                  Yes  ...   
4                  Yes                   No                  Yes  ...   

  PaperlessBilling              PaymentMethod MonthlyCharges  TotalCharges  \
0              Yes           Electronic check          45.65         45.65   
1               No           Electronic check         101.15        385.90   
2               No               Mailed check          20.65        330.60   
3              Yes  Bank transfer (automatic)          43.75        903.60   
4               No    Credit card (automatic)          74.10       5222.30   

   Churn  gender SeniorCitizen  Partner Dependents tenure  
0    Yes    Male             0       No         No      1  
1    Yes    Male             0      Yes        Yes      4  
2     No  Female             1       No         No     17  
3    Yes    Male             0       No         No     22  
4     No  Female             0      Yes        Yes     70  

[5 rows x 21 columns]

In [6]:
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [7]:
data.replace(" ", np.nan, inplace=True)


In [8]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

In [9]:
mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
# Now we know that total charges has nan values
data.isna().sum()



customerID          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
dtype: int64

In [10]:
!python3 -m pip install category_encoders as ce
import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
data_enc.head(1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
joblib.dump(enc, 'CustomerChurnOrdinalEncoder.pkl')
labelled_set = enc.transform(data_enc)
labelled_set.tail(5)

You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


/opt/app-root/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


PhoneService MultipleLines InternetService       OnlineSecurity  \
7038             1            No              No  No internet service   
7039             1           Yes     Fiber optic                   No   
7040             1            No             DSL                  Yes   
7041             1           Yes             DSL                   No   
7042             1            No             DSL                  Yes   

             OnlineBackup     DeviceProtection          TechSupport  \
7038  No internet service  No internet service  No internet service   
7039                   No                   No                   No   
7040                  Yes                   No                   No   
7041                  Yes                  Yes                  Yes   
7042                  Yes                   No                  Yes   

      StreamingTV  StreamingMovies        Contract  PaperlessBilling  \
7038            3                3  Month-to-month                 1   
7039            1                1  Month-to-month                 1   
7040            1                1        One year                 1   
7041            2                2        Two year                 1   
7042            1                2        Two year                 2   

                PaymentMethod  MonthlyCharges  TotalCharges  gender  \
7038             Mailed check           18.85         18.85       1   
7039         Electronic check           74.50         74.50       2   
7040  Credit card (automatic)           53.65       3804.40       1   
7041         Electronic check           84.10       5979.70       1   
7042             Mailed check           71.10        213.35       2   

      SeniorCitizen  Partner  Dependents  tenure  
7038              0        1           1       1  
7039              0        1           1       1  
7040              0        2           1      69  
7041              0        2           2      70  
7042              0        1           1       3

In [11]:

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
data_ohe.head(1)
ohe.fit(data_ohe)
joblib.dump(ohe, 'CustomerChurnOneHotEncoder.pkl')
final_set = ohe.transform(labelled_set)
final_set.tail(5)
labelled_set.shape

/opt/app-root/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(7043, 19)

In [12]:
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Training Data Shape (5634, 36) (5634,)
Testing Data Shape (1409, 36) (1409,)


In [13]:

Y = data['Churn']
X = final_set



In [14]:
def train_and_save_model():
    kfold = KFold(n_splits = 3)
    model = DecisionTreeClassifier(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
    store = get_meta_store()
    model = model.fit(X_train, y_train, run=store)
    joblib.dump(model, 'CustomerChurnPredictor.sav')
    results = model_selection.cross_val_score(model,X,Y,cv = kfold)
    print(results)
    print('Accuracy',results.mean()*100)

    store.log_model(model)
    store.log_metric('Accuracy',results.mean()*100)
    store.log_tag("DecisionTreeClassifier")
    store.log_attribute("data_file_location", "data/full_data_csv/a.csv")
    store.log_attribute("data_file_version", data_file_version)

    return model

In [15]:
from alibi.utils.data import gen_category_map

def explain_model(model, X_train, X_test_record):
    fnames = X_train.columns.tolist()
    predict_fn = lambda x: model.predict_proba(x)
    explainer = AnchorTabular(predict_fn, fnames)
    explainer = explainer.fit(X_train.values, disc_perc=[25, 50, 75])
    explanation = explainer.explain(X_test_record.values[0])
    print('Anchor: %s' % explanation['anchor'])
    print('Precision: %.2f' % explanation['precision'])
    print('Coverage: %.2f' % explanation['coverage'])
    return explainer

In [16]:
minioClient = get_s3_server()
#minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictor.sav' , file_path='./CustomerChurnPredictor.sav')
#minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictorAlibi.dill' , file_path='./CustomerChurnPredictorAlibi.dill')
minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOrdinalEncoder.pkl' , file_path='./CustomerChurnOrdinalEncoder.pkl')
minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOneHotEncoder.pkl' , file_path='./CustomerChurnOneHotEncoder.pkl')


('ecd11095309118779c820f58fdac96dc', None)